### To get an end to end pipeling working

In [64]:
import pandas as pd
import re  ##regex library (used in cleaning + skill extraction)
from sklearn.feature_extraction.text import TfidfVectorizer   ##converts text into numeric vectors (TF-IDF).
from sklearn.metrics.pairwise import cosine_similarity  ##compares vectors and returns similarity scores from 0 to 1

In [66]:
jobs = pd.read_csv(r"C:\Users\jhanvi.kasundra\Downloads\Projects New\Data/jobs_clean.csv")
resumes = pd.read_csv(r"C:\Users\jhanvi.kasundra\Downloads\Projects New\Data/resumes_clean.csv")

In [87]:
# Stable IDs (super important for later Streamlit + Kafka)
jobs = jobs.reset_index(drop=True)
jobs["job_id"] = jobs.index

resumes = resumes.reset_index(drop=True)
resumes["resume_id"] = resumes.index

##After filtering, sampling, dropping duplicates, etc., row numbers change.
## having job id and resume id means you can always track whihc row matched later 

In [89]:
# ============================================================
# BLOCK 2 — Text cleaning (spaces + HTML cleanup)
# ============================================================
def clean_text(s: str) -> str:
    s = str(s)
    s = re.sub(r"<[^>]+>", " ", s)      # remove HTML tags
    s = s.replace("&amp;", "&")         # decode common HTML entity
    s = re.sub(r"\s+", " ", s)          # collapse all whitespace (removes extra spaces/newlines)
    return s.strip()

jobs["job_title"] = jobs["job_title"].fillna("").apply(clean_text)
jobs["job_text"]  = jobs["job_text"].fillna("").apply(clean_text)

resumes["text_cv"] = resumes["text_cv"].fillna("").apply(clean_text)
resumes["category"] = resumes["category"].fillna("unknown").str.lower().str.strip()

## converts to strings safe even if NaN, removes basic html tags like <br>,<p>,etc
## convert &amp; --> &
## colapses multiple spaces/newlines into single space 
##trims ends

In [69]:
# Remove very short/noisy text + duplicates
jobs = jobs.drop_duplicates(subset=["job_title", "job_text"]).copy()
jobs = jobs[jobs["job_text"].str.len() >= 200].copy()

resumes = resumes.drop_duplicates(subset=["category", "text_cv"]).copy()
resumes = resumes[resumes["text_cv"].str.len() >= 200].copy()

print("After cleaning:")
print("jobs:", jobs.shape, "| resumes:", resumes.shape)
## removes duplicate job posts or duplicate resumes inflate results and shlow down compute
## very short text like apply is useless for matching and creates ab dvectors

After cleaning:
jobs: (13510, 6) | resumes: (12353, 3)


In [70]:
# ============================================================
# BLOCK 3 — Global title exclusions (removes junk roles)
# ============================================================
EXCLUDE_TITLES = [
    "data entry", "data-entry", "entry specialist", "data entry assistant",
    "logistics", "warehouse", "driver",
    "facilities", "facility",
    "cashier", "crew member", "barista",
    "car wash", "delivery station",
    "food service", "cook", "server", "dishwasher",
    "security guard", "janitor", "maintenance",
]

exclude_pattern = "|".join([re.escape(x) for x in EXCLUDE_TITLES])

before = len(jobs)
jobs = jobs[~jobs["job_title"].str.lower().str.contains(exclude_pattern, na=False)].copy()
print(f"Jobs after EXCLUDE_TITLES: {jobs.shape} | removed: {before - len(jobs)}")

# Remove academia globally to prevent professor/faculty matches
ACADEMIC_EXCLUDE = r"(?:professor|faculty|lecturer|tenure|university|college|postdoc)"
before = len(jobs)
jobs = jobs[~jobs["job_title"].str.lower().str.contains(ACADEMIC_EXCLUDE, regex=True, na=False)].copy()
print(f"Jobs after ACADEMIC_EXCLUDE: {jobs.shape} | removed: {before - len(jobs)}")

## removes roles you dont want in that bucket (cashier/warehouse/etc)
## makes your final recommendations much more relevant

Jobs after EXCLUDE_TITLES: (13220, 6) | removed: 290
Jobs after ACADEMIC_EXCLUDE: (13160, 6) | removed: 60


In [71]:
# ============================================================
# BLOCK 4 — Category normalization (Exact map + Rule-based fallback)
# Goal: reduce "other" from 6k -> <1k (ideally near 0)
# ============================================================

EXACT_MAP = {
    # HR
    "human resources": "hr",
    "hr": "hr",

    # Banking/Finance
    "accountant": "finance",
    "finance": "finance",
    "banking": "banking",

    # Marketing/media
    "digital media": "digital media",
    "public relations": "marketing",

    # Business / PMO
    "business analyst": "business analyst",
    "pmo": "business analyst",
    "consultant": "consulting",
    "management": "management",
    "operations manager": "management",

    # Tech buckets (explicit)
    "information technology": "information technology",
    "it": "information technology",
    "data science": "data science",
    "database": "information technology",
    "devops": "information technology",
    "etl developer": "information technology",
    "sql developer": "information technology",
    "python developer": "information technology",
    "dotnet developer": "information technology",
    "java developer": "information technology",
    "react developer": "information technology",
    "sap developer": "information technology",
    "testing": "information technology",
    "network security engineer": "information technology",
    "blockchain": "information technology",
}

# Rule-based mapping: checks keywords inside the category string
RULES = [
    # TECH / SOFTWARE / DATA
    (r"(data|ml|machine learning|ai|analytics|business intelligence|bi\b|database|sql|etl|devops|cloud|aws|azure|gcp|python|java|dotnet|react|sap|testing|qa|security|network)", "information technology"),

    # FINANCE / ACCOUNTING
    (r"(finance|financial|account|accountant|audit|tax|treasury|fp&a|risk|credit|bank|banking|insurance)", "finance"),

    # MARKETING / MEDIA / SALES-ish comms
    (r"(marketing|digital|seo|sem|content|social media|brand|advertising|media|public relations|communications|crm)", "marketing"),

    # SALES / BIZDEV
    (r"(sales|business development|bd|account executive|inside sales|customer success|client success)", "sales"),

    # HR / PEOPLE
    (r"(human resources|hr\b|recruit|talent|workforce|people)", "hr"),

    # HEALTHCARE / FITNESS
    (r"(health|healthcare|clinical|hospital|pharma|medical|patient|claims|fitness|wellness|nutrition)", "healthcare"),

    # EDUCATION / ACADEMIA
    (r"(education|teacher|teaching|faculty|professor|lecturer|school|university|college|trainer)", "education"),

    # LEGAL
    (r"(legal|law|advocate|attorney|paralegal|litigation|contract)", "legal"),

    # ENGINEERING (non-software)
    (r"(civil|mechanical|electrical|electronics|aviation|aerospace|automobile|automotive|architecture|construction|building)", "engineering"),

    # DESIGN / CREATIVE
    (r"(design|designer|ui|ux|graphic|web designing|arts|creative)", "design"),

    # AGRICULTURE / ENVIRONMENT
    (r"(agriculture|agri|farming|crop|environment|sustainability)", "agriculture"),

    # BPO / SUPPORT / OPERATIONS
    (r"(bpo|call center|customer service|support|operations|back office)", "operations"),

    # FOOD / HOSPITALITY
    (r"(food|beverage|restaurant|hospitality|chef|cooking)", "hospitality"),

    # APPAREL / FASHION / RETAIL
    (r"(apparel|fashion|retail|merchandising)", "retail"),
]

def normalize_category(cat: str) -> str:
    cat = str(cat).strip().lower()
    if cat in EXACT_MAP:
        return EXACT_MAP[cat]

    # Apply rule-based patterns
    for pattern, label in RULES:
        if re.search(pattern, cat):
            return label

    # If nothing matches, keep as "other"
    return "other"

resumes["category_norm"] = resumes["category"].apply(normalize_category)

print("\nTop normalized categories (after rules):")
print(resumes["category_norm"].value_counts().head(20))

# Debug: see what is still "other" so we can eliminate it further
still_other = resumes.loc[resumes["category_norm"]=="other", "category"].value_counts().head(50)
print("\nTop raw categories still mapped to OTHER:")
print(still_other)

##exact map is direct matches and rules is regex keyword matching 


Top normalized categories (after rules):
category_norm
information technology    3288
engineering               2292
design                     798
finance                    670
management                 657
business analyst           565
education                  388
sales                      348
consulting                 346
digital media              342
retail                     317
marketing                  314
hr                         305
healthcare                 297
banking                    292
legal                      282
data science               275
agriculture                228
operations                 197
hospitality                152
Name: count, dtype: int64

Top raw categories still mapped to OTHER:
Series([], Name: count, dtype: int64)


In [76]:
resumes["category_norm"].value_counts().get("other", 0)

0

In [78]:
# ============================================================
# BLOCK 5 — Matching (TF-IDF embeddings + cosine similarity)
# ============================================================

CATEGORY_FILTERS = {
    # Finance / Banking
    "banking": r"(?:bank|banking|finance|financial|credit|risk|fraud|loan|mortgage|asset management|trading|quant|portfolio|compliance|model risk|treasury)",
    "finance": r"(?:finance|financial|accounting|audit|tax|fp&a|portfolio|investment|trading|equity|fixed income|credit|risk|treasury|compliance)",

    # Marketing / Media
    "digital media": r"(?:marketing|seo|sem|paid media|campaign|ads|advertising|google analytics|social media|content|brand|ecommerce|crm)",

    # Tech
    "information technology": r"(?:software|cloud|aws|azure|gcp|devops|data engineer|etl|pipeline|kafka|spark|api|microservices|ci/cd|docker|kubernetes|ml|llm|nlp)",

    # HR
    "hr": r"(?:recruiting|talent|people analytics|hr analytics|compensation|benefits|workforce|employee|hrbp)",

    # Business analyst / PMO
    "business analyst": r"(?:requirements|stakeholder|business analysis|product|roadmap|user stories|agile|scrum|kpi|dashboards|reporting|analytics)",

    # Management (optional)
    "management": r"(?:operations|process improvement|strategy|planning|budget|forecast|kpi|stakeholder|program management|project management)",

    # Engineering
    "engineering": r"(?:engineering|civil|mechanical|electrical|construction|architecture|automotive|aviation|aerospace|manufacturing|plant|systems)",

    # Design
    "design": r"(?:design|designer|ui|ux|graphic|visual|product design|web design|creative)",

    # Sales
    "sales": r"(?:sales|business development|account executive|pipeline|quota|crm|customer success|client success)",

    # Consulting
    "consulting": r"(?:consultant|consulting|advisory|client|engagement|stakeholder|strategy)",

    # Healthcare
    "healthcare": r"(?:health|healthcare|clinical|hospital|pharma|medical|patient|claims|ehr|emr|hipaa)",

    # Legal
    "legal": r"(?:legal|law|advocate|attorney|paralegal|litigation|contract)",

    # Operations
    "operations": r"(?:operations|support|call center|customer service|back office|process|workflow)",

    # Hospitality
    "hospitality": r"(?:hospitality|restaurant|food|beverage|chef|kitchen|cooking|hotel)",

    # Retail
    "retail": r"(?:retail|merchandising|store|apparel|fashion|ecommerce|inventory)",

    # Education
    "education": r"(?:education|teacher|teaching|school|trainer|curriculum|student)",

    # Agriculture
    "agriculture": r"(?:agriculture|farming|crop|agri|soil|yield|sustainability)",
}

## group resumes by bucket and then filters jobs relevant to that bucket and then run tf-idf only inside that smaller pool making it faster and more accurate 

In [80]:
results = []
top_k = 5
MAX_JOBS_PER_CATEGORY = 5000  # set None to disable (slower)
#### griup all reusmes in that bucket
for cat, group in resumes.groupby("category_norm"):
    print("\n===========================")
    print(f"Category bucket: {cat} | resumes: {len(group)} | jobs before filter: {len(jobs)}")

    jobs_cat = jobs.copy()

    # Apply category job-text filter if available
    if cat in CATEGORY_FILTERS:
        pattern = CATEGORY_FILTERS[cat]
        before_cat = len(jobs_cat)
        jobs_cat = jobs_cat[jobs_cat["job_text"].str.lower().str.contains(pattern, regex=True, na=False)].copy()
        print(f"Jobs after category TEXT filter: {len(jobs_cat)} | removed: {before_cat - len(jobs_cat)}")
    else:
        print("No category filter found -> using global pool")

    # Fallback if too few jobs -- so you dont end up matching against tiny pool
    if len(jobs_cat) < 300:
        print("Too few jobs after filtering -> fallback to global pool.")
        jobs_cat = jobs.copy()

    # Optional cap for speed
    if MAX_JOBS_PER_CATEGORY is not None and len(jobs_cat) > MAX_JOBS_PER_CATEGORY:
        jobs_cat = jobs_cat.sample(MAX_JOBS_PER_CATEGORY, random_state=42).copy()   ### keeps run time manageable 
        print(f"Capped jobs to: {len(jobs_cat)}")

    # TF-IDF embeddings  -- you fit tf -idf once on combined text so resumes and jobs share same vocab + vector space 
    vectorizer = TfidfVectorizer(
        stop_words="english",
        max_features=60000,
        ngram_range=(1, 2),
        min_df=2
    )

    combined = pd.concat([jobs_cat["job_text"], group["text_cv"]], ignore_index=True)
    X = vectorizer.fit_transform(combined)
### first part is jobs and second part is resumes 
    job_vecs = X[:len(jobs_cat)]
    res_vecs = X[len(jobs_cat):]

    # Similarity scores-- produces a matrix rows = resumes in this bucket ; cols = jobs in thi bucket; val = similarity score (0 to 1)
    sim = cosine_similarity(res_vecs, job_vecs)

    group = group.reset_index(drop=True)

    for i in range(len(group)):
        best = sim[i].argsort()[::-1][:top_k] ###argsort() gives job indices sorted by score ##[::-1] reverses (highest first) ##[:5] takes top 5
        for rank, j in enumerate(best, start=1):
            results.append({
                "resume_id": int(group.loc[i, "resume_id"]),
                "resume_category_raw": group.loc[i, "category"],
                "resume_category_norm": cat,
                "rank": rank,
                "score": float(sim[i, j]),
                "job_id": int(jobs_cat.iloc[j]["job_id"]),
                "job_title": jobs_cat.iloc[j]["job_title"],
                "company_name": jobs_cat.iloc[j].get("company_name", None),
                "location": jobs_cat.iloc[j].get("location", None),
                "source": jobs_cat.iloc[j]["source"],
            })

## store results  you append rows like resume id , cat. rank score, job id , job title, cmpany, loctm, source 


Category bucket: agriculture | resumes: 228 | jobs before filter: 13160
Jobs after category TEXT filter: 426 | removed: 12734

Category bucket: banking | resumes: 292 | jobs before filter: 13160
Jobs after category TEXT filter: 7630 | removed: 5530
Capped jobs to: 5000

Category bucket: business analyst | resumes: 565 | jobs before filter: 13160
Jobs after category TEXT filter: 11748 | removed: 1412
Capped jobs to: 5000

Category bucket: consulting | resumes: 346 | jobs before filter: 13160
Jobs after category TEXT filter: 8150 | removed: 5010
Capped jobs to: 5000

Category bucket: data science | resumes: 275 | jobs before filter: 13160
No category filter found -> using global pool
Capped jobs to: 5000

Category bucket: design | resumes: 798 | jobs before filter: 13160
Jobs after category TEXT filter: 12772 | removed: 388
Capped jobs to: 5000

Category bucket: digital media | resumes: 342 | jobs before filter: 13160
Jobs after category TEXT filter: 5560 | removed: 7600
Capped jobs to:

In [82]:
out = pd.DataFrame(results).sort_values(["resume_id", "rank"])
out.to_csv(r"C:\Users\jhanvi.kasundra\Downloads\Projects New\Data\matches_tfidf_all.csv", index=False)
## save matches 
print("\nSaved matches file.")
print(out.head(15))


Saved matches file.
       resume_id resume_category_raw resume_category_norm  rank     score  \
27630          0          accountant              finance     1  0.105080   
27631          0          accountant              finance     2  0.098859   
27632          0          accountant              finance     3  0.088279   
27633          0          accountant              finance     4  0.080278   
27634          0          accountant              finance     5  0.071238   
27635          1          accountant              finance     1  0.150918   
27636          1          accountant              finance     2  0.127709   
27637          1          accountant              finance     3  0.107706   
27638          1          accountant              finance     4  0.107021   
27639          1          accountant              finance     5  0.104937   
27640          2          accountant              finance     1  0.110799   
27641          2          accountant              finan

In [85]:
# ============================================================
# BLOCK 6 — Skill Extraction + Skill Gap (adds explainability)
# ============================================================

# A starter skills list (expand anytime)
SKILLS = [
    "python","sql","excel","tableau","power bi","spss","r","sas",
    "aws","azure","gcp","snowflake","databricks","spark","hadoop","kafka","airflow","dbt",
    "pandas","numpy","scikit-learn","tensorflow","pytorch",
    "nlp","bert","llm","prompt engineering",
    "statistics","regression","classification","time series","forecasting",
    "ab testing","experiment","etl","data pipeline","data modeling","dimensional modeling",
    "git","ci/cd","docker","kubernetes"
]

def extract_skills(text: str) -> set:
    text_l = str(text).lower()
    found = set()
    for s in SKILLS:
        # word boundary-ish match; works fine for most skill tokens
        pattern = r"\b" + re.escape(s.lower()) + r"\b"
        if re.search(pattern, text_l):
            found.add(s)
    return found

# Load matches
matches = pd.read_csv(r"C:\Users\jhanvi.kasundra\Downloads\Projects New\Data\matches_tfidf_all.csv")

# Create quick lookup tables for full text
jobs_lookup = jobs.set_index("job_id")
res_lookup = resumes.set_index("resume_id")

rows = []
for _, m in matches.iterrows():
    rid = int(m["resume_id"])
    jid = int(m["job_id"])

    resume_text = res_lookup.loc[rid, "text_cv"]
    job_text = jobs_lookup.loc[jid, "job_text"]

    r_sk = extract_skills(resume_text)
    j_sk = extract_skills(job_text)

    rows.append({
        "resume_id": rid,
        "job_id": jid,
        "rank": int(m["rank"]),
        "score": float(m["score"]),
        "resume_category_norm": m["resume_category_norm"],
        "job_title": m["job_title"],
        "resume_skills": ", ".join(sorted(r_sk)),
        "job_skills": ", ".join(sorted(j_sk)),
        "missing_skills": ", ".join(sorted(j_sk - r_sk)),
        "missing_skills_count": len(j_sk - r_sk)
    })

skill_gap_df = pd.DataFrame(rows)
skill_gap_df.to_csv(r"C:\Users\jhanvi.kasundra\Downloads\Projects New\Data\matches_with_skill_gap.csv", index=False)

print("Saved: matches_with_skill_gap.csv")
print(skill_gap_df.head(10))

Saved: matches_with_skill_gap.csv
   resume_id  job_id  rank     score resume_category_norm  \
0          0    8560     1  0.105080              finance   
1          0   11710     2  0.098859              finance   
2          0   14076     3  0.088279              finance   
3          0    1232     4  0.080278              finance   
4          0    5437     5  0.071238              finance   
5          1   13525     1  0.150918              finance   
6          1    7151     2  0.127709              finance   
7          1    2619     3  0.107706              finance   
8          1    5998     4  0.107021              finance   
9          1     374     5  0.104937              finance   

                                           job_title resume_skills  \
0                          Data Scientist - Insights           sql   
1  Sr. to Lead Data Scientist - AML / Anti-Money ...           sql   
2                   Research & Instruction Librarian           sql   
3             